# Feature engineering and ETL
In the initial data exploration it was established that the data is skewed , has some outliers and missing values for numerical columns. There are categorical features that need to be one hot encoded. Since multiple models will be used multiple saves will be created depending on the transformations. For eg, Tree based models do not require that data be scaled while it is a good idea to scale data for neural networks.

In total 3 different ETL pipelines are created. One with one hot encoded categorical columns, one with scaled columns and one with scaled and transformed. The Yeo jhonson transformation is used on numerical columns to handle skewness. this should work well with Neural Network.

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, PowerTransformer

In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/gagan3012/online-shoppers-intention-/master/online_shoppers_intention.csv")
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0.0,0.0,0.0,0.0,2.0,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0.0,-1.0,0.0,-1.0,1.0,-1.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0.0,0.0,0.0,0.0,2.0,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0.0,0.0,0.0,0.0,10.0,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [3]:
df['Administrative']=df['Administrative'].clip(lower=0,upper=20)
df['PageValues']=df['PageValues'].clip(lower=0,upper=250)
df['Administrative_Duration']=df['Administrative_Duration'].clip(lower=0,upper=1500)
df['Informational']=df['Informational'].clip(lower=0,upper=10)
df['ProductRelated_Duration']=df['ProductRelated_Duration'].clip(lower=0,upper=20000)
df['ProductRelated']=df['ProductRelated'].clip(lower=0,upper=400)
df['Informational_Duration']=df['Informational_Duration'].clip(lower=0,upper=1500)

In [4]:
cat_dtype = pd.api.types.CategoricalDtype(categories=['Feb', 'Mar', 'May', 'June', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov','Dec'],
                                          ordered=True)#ordered dtype for month
df['Month']=df['Month'].astype(cat_dtype)


In [5]:
df.shape

(12330, 18)

In [7]:
df.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.313901,79.909572,0.500568,33.929274,31.630318,1187.694533,0.022152,0.043003,5.863980,0.061427,2.124006,2.357097,3.147364,4.069586
std,3.299558,165.983561,1.235324,133.387972,43.102423,1769.998361,0.048427,0.048527,18.173049,0.198917,0.911325,1.717277,2.401591,4.025169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,185.000000,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000
50%,1.000000,8.000000,0.000000,0.000000,18.000000,599.766190,0.003119,0.025124,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000
75%,4.000000,93.500000,0.000000,0.000000,38.000000,1466.479902,0.016684,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000
max,20.000000,1500.000000,10.000000,1500.000000,400.000000,20000.000000,0.200000,0.200000,250.000000,1.000000,8.000000,13.000000,9.000000,20.000000


In [8]:
df.dtypes

Administrative              float64
Administrative_Duration     float64
Informational               float64
Informational_Duration      float64
ProductRelated              float64
ProductRelated_Duration     float64
BounceRates                 float64
ExitRates                   float64
PageValues                  float64
SpecialDay                  float64
Month                      category
OperatingSystems              int64
Browser                       int64
Region                        int64
TrafficType                   int64
VisitorType                  object
Weekend                        bool
Revenue                        bool
dtype: object

In [9]:
cat_cols=['Weekend','VisitorType','Region','Browser','OperatingSystems','TrafficType']
df[cat_cols]=df[cat_cols].astype('category')
df =df.dropna()

In [10]:
df_encoded=pd.get_dummies(df,columns=cat_cols+['Month'],prefix=cat_cols+['Month'])

In [11]:
numeric_cols = df.select_dtypes(include='float64').columns

In [32]:
scaler = MinMaxScaler()
pt=PowerTransformer()
df_transformed = df_encoded.copy()
df_scaled = df_encoded.copy()
df_scaled[numeric_cols]= scaler.fit_transform(df_encoded[numeric_cols])
df_transformed[numeric_cols]= pt.fit_transform(df_encoded[numeric_cols])

In [33]:
df_scaled.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Feb,Month_Mar,Month_May,Month_June,Month_Jul,Month_Aug,Month_Sep,Month_Oct,Month_Nov,Month_Dec
0,0.0,0.0,0.0,0.0,0.0025,0.000000,1.00,1.00,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0050,0.003200,0.00,0.50,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0025,0.000000,1.00,1.00,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0050,0.000133,0.25,0.70,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0250,0.031375,0.10,0.25,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0


In [34]:
df_transformed.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Feb,Month_Mar,Month_May,Month_June,Month_Jul,Month_Aug,Month_Sep,Month_Oct,Month_Nov,Month_Dec
0,-0.991065,-0.997544,-0.52074,-0.491916,-1.923024,-2.094361,2.207184,1.983626,-0.529787,-0.336232,...,1,0,0,0,0,0,0,0,0,0
1,-0.991065,-0.997544,-0.52074,-0.491916,-1.576413,-1.078325,-0.799476,1.571869,-0.529787,-0.336232,...,1,0,0,0,0,0,0,0,0,0
2,-0.991065,-0.997544,-0.52074,-0.491916,-1.923024,-2.094361,2.207184,1.983626,-0.529787,-0.336232,...,1,0,0,0,0,0,0,0,0,0
3,-0.991065,-0.997544,-0.52074,-0.491916,-1.576413,-1.875374,1.711008,1.833617,-0.529787,-0.336232,...,1,0,0,0,0,0,0,0,0,0
4,-0.991065,-0.997544,-0.52074,-0.491916,-0.445603,0.054466,0.758331,0.724766,-0.529787,-0.336232,...,1,0,0,0,0,0,0,0,0,0


In [35]:
df_encoded.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Feb,Month_Mar,Month_May,Month_June,Month_Jul,Month_Aug,Month_Sep,Month_Oct,Month_Nov,Month_Dec
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,2.0,64.000000,0.00,0.10,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,2.0,2.666667,0.05,0.14,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,10.0,627.500000,0.02,0.05,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
